In [1]:
import sys

path_to_add = "../gloabl_files/ml_binary_classification_gridsearch_hyperOpt/"
sys.path.append(path_to_add)
path_to_add = "../"
sys.path.append(path_to_add)


In [ ]:
import os
import subprocess

# Boolean flag to control CPU core binding
limit_cpu_cores = False

if limit_cpu_cores:
    # Get the current process ID
    pid = os.getpid()
    print(f"Notebook PID: {pid}")

    # Define the CPU cores to bind (e.g., cores 0-3)
    core_range = "0-3"

    # Use taskset to bind the current process to specific CPU cores
    try:
        # Execute taskset command
        subprocess.run(["taskset", "-cp", core_range, str(pid)], check=True)
        print(f"Successfully bound Notebook PID {pid} to CPU cores {core_range}.")
    except FileNotFoundError:
        print("Error: 'taskset' command not found. Please ensure it is installed.")
    except subprocess.CalledProcessError as e:
        print(f"Error while setting CPU affinity: {e}")
else:
    print("CPU core binding is disabled.")


In [3]:

# import subprocess

# with open('requirements.txt', 'r') as file:
#     packages = file.readlines()

# for package in packages:
#     package = package.strip()  # Remove leading/trailing whitespaces, if any
#     try:
#         subprocess.check_call(["pip", "install", package])
#         print(f"Successfully installed {package}")
#     except Exception as e:
#         print(f"Failed to install {package}: {e}")


In [ ]:
pwd

In [5]:
import sys

In [6]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
pwd

In [8]:
from ml_grid.util import grid_param_space

In [9]:
import ipywidgets as ipw
output = ipw.Output()

In [10]:
import warnings
warnings.filterwarnings('ignore') 

In [11]:
import os
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

In [12]:
#%pip install hyperopt

In [13]:
from hyperopt import fmin, tpe, hp

In [14]:
grid = {
            
            'resample' : ['undersample', 'oversample', None],
            'scale'    : [True, False],
            'feature_n': [100, 95, 75, 50, 25, 5],
            'param_space_size':['medium', 'xsmall'],
            'n_unique_out': [10],
            'outcome_var_n':['1'],
                            'percent_missing':[99, 95, 80],  #n/100 ex 95 for 95% # 99.99, 99.5, 9
                            'corr':[0.98, 0.85, 0.5, 0.25],
                            'data':[{'age':[True, False],
                                    'sex':[True, False],
                                    'bmi':[True],
                                    'ethnicity':[True, False],
                                    'bloods':[True, False],
                                    'diagnostic_order':[True, False],
                                    'drug_order':[True, False],
                                    'annotation_n':[True, False],
                                    'meta_sp_annotation_n':[True, False],
                                    'annotation_mrc_n':[True, False],
                                    'meta_sp_annotation_mrc_n':[True, False],
                                    'core_02':[False],
                                    'bed':[False],
                                    'vte_status':[True],
                                    'hosp_site':[True],
                                    'core_resus':[False],
                                    'news':[False],
                                    'date_time_stamp':[ False]
                                    
                                    }]
        }

In [15]:
from hyperopt import hp
from hyperopt.pyll import scope

space = {
    'resample': hp.choice('resample', ['undersample', 'oversample', None]),
    'scale': hp.choice('scale', [True, False]),
    'feature_n': hp.choice('feature_n', [100, 95, 75, 50, 25, 5]),
    'param_space_size': hp.choice('param_space_size', ['medium', 'xsmall']),
    'n_unique_out': hp.choice('n_unique_out', [10]),
    'outcome_var_n': hp.choice('outcome_var_n', ['1']),
    'percent_missing': hp.choice('percent_missing', [99, 95, 80]),
    'corr': hp.choice('corr', [0.98, 0.85, 0.5, 0.25]),
    'feature_selection_method': hp.choice('feature_selection_method', ['anova', 'markov_blanket']),
    'data': {
        'age': hp.choice('age', [True, False]),
        'sex': hp.choice('sex', [True, False]),
        'bmi': hp.choice('bmi', [True]),
        'ethnicity': hp.choice('ethnicity', [True, False]),
        'bloods': hp.choice('bloods', [True, False]),
        'diagnostic_order': hp.choice('diagnostic_order', [True, False]),
        'drug_order': hp.choice('drug_order', [True, False]),
        'annotation_n': hp.choice('annotation_n', [True, False]),
        'meta_sp_annotation_n': hp.choice('meta_sp_annotation_n', [True, False]),
        'annotation_mrc_n': hp.choice('annotation_mrc_n', [True, False]),
        'meta_sp_annotation_mrc_n': hp.choice('meta_sp_annotation_mrc_n', [True, False]),
        'core_02': hp.choice('core_02', [False]),
        'bed': hp.choice('bed', [False]),
        'vte_status': hp.choice('vte_status', [True]),
        'hosp_site': hp.choice('hosp_site', [True]),
        'core_resus': hp.choice('core_resus', [False]),
        'news': hp.choice('news', [False]),
        'date_time_stamp': hp.choice('date_time_stamp', [False]),
        'appointments': hp.choice('appointments', [False])
    }
}


In [16]:
# Breast cancer sample space:

space_breast_cancer = {
    'resample': hp.choice('resample', ['undersample', 'oversample', None]),
    'scale': hp.choice('scale', [True, False]),
    'feature_n': hp.choice('feature_n', [ 25, 5]),
    'param_space_size': hp.choice('param_space_size', ['medium', 'xsmall']),
    'n_unique_out': hp.choice('n_unique_out', [10]),
    'outcome_var_n': hp.choice('outcome_var_n', ['1']), # Optimise for alternate representations of outcome variable.
    'percent_missing': hp.choice('percent_missing', [99, 95, 80]),
    'corr': hp.choice('corr', [0.98, 0.85, 0.5, 0.25]),
    'data': {
        'age': hp.choice('age', [False]),
        'sex': hp.choice('sex', [ False]),
        'bmi': hp.choice('bmi', [False]),
        'ethnicity': hp.choice('ethnicity', [ False]),
        'bloods': hp.choice('bloods', [True, ]),
        'diagnostic_order': hp.choice('diagnostic_order', [ False]),
        'drug_order': hp.choice('drug_order', [ False]),
        'annotation_n': hp.choice('annotation_n', [ False]),
        'meta_sp_annotation_n': hp.choice('meta_sp_annotation_n', [ False]),
        'annotation_mrc_n': hp.choice('annotation_mrc_n', [ False]),
        'meta_sp_annotation_mrc_n': hp.choice('meta_sp_annotation_mrc_n', [ False]),
        'core_02': hp.choice('core_02', [False]),
        'bed': hp.choice('bed', [False]),
        'vte_status': hp.choice('vte_status', [False]),
        'hosp_site': hp.choice('hosp_site', [False]),
        'core_resus': hp.choice('core_resus', [False]),
        'news': hp.choice('news', [False]),
        'date_time_stamp': hp.choice('date_time_stamp', [False]),
    }
}

#### Setup the logger

In [17]:
from ml_grid.util.logger_setup import setup_logger
import logging

logger = logging.getLogger('matplotlib.font_manager')

# Set the logging level to suppress debug messages
logger.setLevel(logging.INFO)

In [18]:
# Optionally exclude model classes

model_class_dict = {
        "LogisticRegression_class": True,
        "knn_classifiers_class": True,
        "quadratic_discriminant_analysis_class": True,
        "SVC_class": True,
        "XGB_class_class": True,
        "mlp_classifier_class": True,
        "RandomForestClassifier_class": True,
        "GradientBoostingClassifier_class": True,
        "CatBoost_class": True,
        "GaussianNB_class": True,
        "LightGBMClassifierWrapper": True,
        "adaboost_class": True,
        "kerasClassifier_class": True,
        "knn__gpu_wrapper_class": True,
        "NeuralNetworkClassifier_class": True,
        "TabTransformer_class": False,
    }

In [ ]:
import ml_grid
import pathlib
import datetime
from tqdm import tqdm
import random
from IPython.display import clear_output
import pandas as pd

from ml_grid.model_classes.h2o_classifier_class import h2o_classifier_class
from ml_grid.util.project_score_save import project_score_save_class

from ml_grid.pipeline.data import pipe

from ml_grid.util.param_space import ParamSpace

random.seed(1234)


base_project_dir_global = 'HFE_ML_experiments/'

logger = setup_logger(log_folder_path = base_project_dir_global)

# Create a logger
logger = logging.getLogger(__name__)

# Add a filter to exclude logs not related to numba.core.byteflow
class ByteflowFilter(logging.Filter):
    def filter(self, record):
        return record.name.startswith('numba.core.byteflow')

# Add the filter to the logger
logger.addFilter(ByteflowFilter())

pathlib.Path(base_project_dir_global).mkdir(parents=True, exist_ok=True) 

st_time = datetime.datetime.now().strftime("%Y-%m-%d_%I-%M-%S_%p")

base_project_dir = 'HFE_ML_experiments/' + st_time + "/"
additional_naming = "HFE_ML_Grid_"

print(base_project_dir)

pathlib.Path(base_project_dir).mkdir(parents=True, exist_ok=True) 

input_csv_path = 'breast_cancer_dataset.csv'

#input_csv_path = 'test_data_hfe_1yr_m_small_multiclass.csv'

#input_csv_path = os.path.join('..', 'gloabl_files', 'ml_binary_classification_gridsearch_hyperOpt', 'notebooks' ,'test_data_hfe_1yr_m_small.csv') #large

#init csv to store each local projects results

project_score_save_class(base_project_dir)

n_iter = 1000

grid_iter_obj = grid_param_space.Grid(sample_n=n_iter).settings_list_iterator


def objective(local_param_dict, outcome_var=None):
    clear_output()
    #get settings from iterator over grid of settings space
    #local_param_dict = next(grid_iter_obj)
    print(local_param_dict)
    
    #init random number string
    
    idx = random.randint(0,999999999999999999999)

    #create object from settings
    ml_grid_object = pipe(input_csv_path,
                                                drop_term_list=['chrom', 'hfe', 'phlebo'],
                                                local_param_dict=local_param_dict,
                                                base_project_dir = base_project_dir,
                                                additional_naming = additional_naming,
                                                test_sample_n = 0,
                                                param_space_index = idx,
                                                model_class_dict = model_class_dict,
                                                outcome_var_override = outcome_var
                                                #outcome_var_override = None #override outcome var, example = 'outcome_var_myeloma'
                                                #outcome_var_override = outcome_var_list[outcome_index] # set if multi class
                                                )

    from ml_grid.pipeline import main
    
    # from ml_grid.model_classes.h2o_classifier_class import h2o_classifier_class

    # Example overwrite/append model_class list
    # temp_param_space_size = ParamSpace(ml_grid_object.local_param_dict.get("param_space_size"))

    # ml_grid_object.model_class_list = [h2o_classifier_class(
    #             X=ml_grid_object.X_train,
    #             y=ml_grid_object.y_train,
    #             parameter_space_size=temp_param_space_size,
    #         )]

    # Example append 
    # if(ml_grid_object.time_series_mode == False):
    #temp_param_space_size = ParamSpace(ml_grid_object.local_param_dict.get("param_space_size"))

    #     ml_grid_object.model_class_list.extend([h2o_classifier_class(
    #                 X=ml_grid_object.X_train,
    #                 y=ml_grid_object.y_train,
    #                 parameter_space_size=temp_param_space_size,
    #             )])

    #pass object to be evaluated and write results to csv
    errors, highest_score = main.run(ml_grid_object, local_param_dict=local_param_dict).execute()
    
    
    
    results_df = pd.read_csv(base_project_dir + 'final_grid_score_log.csv')
    
    #highest_metric_from_run = results_df[results_df['i'] == str(idx)].sort_values(by='auc')['auc'].iloc[-1]
    
    highest_metric_from_run = highest_score # for hyperopt multi procesess #AUC
    
    display(results_df[results_df['i'] == str(idx)].sort_values(by='auc').iloc[0])
    
    result = {
        "loss": 1-float(highest_metric_from_run),
        "status": "ok"  # Indicate that the evaluation was successful
    }
    return result
    
     

In [20]:
#objective(next(grid_iter_obj))

In [ ]:
print("done")

In [22]:
# results_df = pd.read_csv(base_project_dir + 'final_grid_score_log.csv')
    
# highest_metric_from_run = results_df[results_df['i'] == str(900424809465212743016)].sort_values(by='auc')['auc'].iloc[-1]

# highest_metric_from_run

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [24]:
trials = Trials()

In [25]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials = trials,
           verbose=1
           )

KeyboardInterrupt: 

In [ ]:
best

In [ ]:
results_df = pd.read_csv(base_project_dir + 'final_grid_score_log.csv')

In [ ]:
results_df.sort_values('auc', ascending=False).iloc[0]

In [ ]:
results_df.sort_values('auc', ascending=False)

In [ ]:
break

In [ ]:
import pandas as pd
dft = pd.read_csv('test_data_hfe_1yr_m_small_multiclass.csv', nrows=1)
dft

In [ ]:
# get outcome variables by finding prefix "outcome_var_" in column list

outcome_var_list = [dft.columns[i] for i in range(len(dft.columns)) if "outcome_var_" in dft.columns[i]]

outcome_var_list

In [29]:
# Multiple outcomes one vs rest

In [ ]:
import threading
from concurrent.futures import ThreadPoolExecutor, TimeoutError, as_completed
from hyperopt import fmin, tpe, Trials
import time
from datetime import datetime
from sklearn.utils.parallel import delayed, Parallel
from joblib import parallel_backend

def process_single_outcome(outcome_index, outcome_var_list):
    """Process a single outcome index"""
    outcome_var = outcome_var_list[outcome_index]
    thread_name = threading.current_thread().name
    start_time = datetime.now()

    print(f"[{start_time}] Thread: {thread_name} - Starting outcome {outcome_index}: {outcome_var}")

    # Wrap objective to include the outcome_var and handle parallel backend
    def objective_with_outcome(params):
        print(f"Thread: {thread_name} - Evaluating params: {params} for outcome {outcome_var}")
        # Use threading backend for scikit-learn operations
        with parallel_backend('threading', n_jobs=1):
            return objective(params, outcome_var)

    try:
        # Configure scikit-learn to use threading backend
        with parallel_backend('threading', n_jobs=1):
            best = fmin(
                fn=objective_with_outcome,
                space=space,
                algo=tpe.suggest,
                max_evals=10,
                trials=Trials(),
                verbose=0
            )
        end_time = datetime.now()
        print(f"[{end_time}] Thread: {thread_name} - Finished outcome {outcome_index} (Duration: {end_time - start_time})")
        return outcome_index, best, None
    except Exception as e:
        print(f"Error in {thread_name} for outcome {outcome_var}: {str(e)}")
        return outcome_index, None, str(e)

start_total = datetime.now()
print(f"Starting all optimizations at {start_total}")

# Use threading backend globally
with parallel_backend('threading', n_jobs=80):
    # Run parallel processing using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=19) as executor:  # One thread per outcome
        futures = [executor.submit(process_single_outcome, i, outcome_var_list) for i in range(19)]

        print(f"Submitted {len(futures)} tasks to parallel executor")

        # Process results as they complete
        for future in as_completed(futures):
            outcome_index = futures.index(future)
            try:
                _, best, error = future.result(timeout=360*2)

                if error:
                    print(f"Exception on fmin on {outcome_var_list[outcome_index]}: {error}")
                elif best is not None:
                    print(f"Best parameters for {outcome_var_list[outcome_index]}: {best}")
                else:
                    print(f"No result for {outcome_var_list[outcome_index]}")

            except TimeoutError:
                print(f"Timeout reached for {outcome_var_list[outcome_index]}")

end_total = datetime.now()
print(f"\nCompleted all optimizations at {end_total}")
print(f"Total duration: {end_total - start_total}")

In [ ]:
break